In [1]:
import cv2
import os
import shutil
import matplotlib.pyplot as plt
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Updated import
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [2]:
import os
from sklearn.model_selection import train_test_split

In [3]:
image_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Apple DS"  # Replace with your actual image directory
class_names = ["Apple_BR", "Apple_CR", "Apple_HE"]  # Replace with your actual class names

In [4]:
import os
import numpy as np
from PIL import Image

# Initialize empty lists for storing image paths and labels
image_paths = []
labels = []

# Loop through each class directory
for class_name in class_names:
    class_dir = os.path.join(image_dir, class_name)
    for image_path in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, image_path))
        labels.append(class_name)       

In [5]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, 
                                                    test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, random_state=42)

In [6]:
print("Length of X_train:", len(X_train))
print("Length of X_test:", len(X_test))
print("Length of X_val:", len(X_val))

Length of X_train: 172
Length of X_test: 54
Length of X_val: 44


In [7]:
import os

train_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Train"  # Replace with your desired path
test_dir = r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Test"  # Replace with your desired path
val_dir =  r"C:\Users\laptop\AI Huawei\26_8_2024\Splitting DS\Valid"  # Replace with your desired path

In [8]:
for image_path, label in zip(X_val, y_val):
    class_dir = os.path.join(val_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [9]:
for image_path, label in zip(X_train, y_train):
    class_dir = os.path.join(train_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [10]:
for image_path, label in zip(X_test, y_test):
    class_dir = os.path.join(test_dir, label)  # Construct class-specific subdirectory
    os.makedirs(class_dir, exist_ok=True)  # Create class directory if it doesn't exist
    shutil.copy(image_path, class_dir)  # Copy image to the class directory

In [11]:
# dimensions of our images.
img_width, img_height = 224, 224
batch_size = 5  

train_datagen = ImageDataGenerator(
        rescale=1./255,            # normalize pixel values to [0,1]
        shear_range=0.2,      
        zoom_range=0.2,    
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)  


val_datagen = ImageDataGenerator(
         rescale=1./255)       # normalize pixel values to [0,1]

import numpy
numpy.random.seed(42)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 240 images belonging to 3 classes.
Found 83 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Define the input shape
input_shape = (224, 224, 3)

# Load the DenseNet121 model pretrained on ImageNet, specifying the input shape
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Build the model
model = Sequential([
    Input(shape=input_shape),
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=20,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=5
)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/5


C:\Users\laptop\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 34s 508ms/step - accuracy: 0.4373 - loss: 12.8237 - val_accuracy: 0.3600 - val_loss: 15.9661
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 433ms/step - accuracy: 0.5806 - loss: 8.6804 - val_accuracy: 0.2800 - val_loss: 10.9161
Epoch 3/5
 8/20 ━━━━━━━━━━━━━━━━━━━━ 4s 407ms/step - accuracy: 0.8978 - loss: 6.0818

C:\ProgramData\Anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - accuracy: 0.8691 - loss: 6.0626 - val_accuracy: 0.6800 - val_loss: 6.1979
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 443ms/step - accuracy: 0.8717 - loss: 5.4916 - val_accuracy: 0.6250 - val_loss: 5.9538
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 436ms/step - accuracy: 0.7749 - loss: 4.4281 - val_accuracy: 0.6000 - val_loss: 5.1575


In [13]:
from sklearn.preprocessing import LabelEncoder

# Use LabelEncoder to encode 'no' as 0 and 'yes' as 1
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

# Print the encoded labels
print("Encoded Labels:")
print(y_test_encoded)

Encoded Labels:
[0 1 0 1 2 1 2 0 1 2 2 2 2 0 1 2 2 1 1 0 2 2 0 0 2 0 0 2 1 0 1 0 1 2 2 0 1
 0 1 1 0 1 2 0 0 2 1 2 1 2 0 1 2 2]


In [14]:
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 102 images belonging to 3 classes.


In [15]:
model.evaluate(test_data)

 2/21 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.7500 - loss: 5.4942 

C:\Users\laptop\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.6585 - loss: 5.1353


[5.07850980758667, 0.6274510025978088]

In [16]:
import numpy as np

# Assuming test_data is a tf.data.Dataset or similar that yields (input, label) pairs
#y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred_prob = model.predict(test_data)
y_pred = np.argmax(y_pred_prob, axis=1)  # for multi-class classification
y_pred_prob

21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 145ms/step


array([[1.68146584e-02, 4.14609938e-04, 9.82770801e-01],
       [3.50045204e-01, 8.80610850e-03, 6.41148627e-01],
       [9.85287309e-01, 2.63015835e-07, 1.47124762e-02],
       [4.98954467e-02, 7.31459249e-06, 9.50097263e-01],
       [9.97483552e-01, 1.62891695e-07, 2.51641148e-03],
       [9.86273110e-01, 1.68038156e-07, 1.37268035e-02],
       [7.61562437e-02, 1.65182911e-03, 9.22191918e-01],
       [8.74068916e-01, 5.09924860e-07, 1.25930533e-01],
       [3.02891731e-01, 1.29908722e-06, 6.97106957e-01],
       [3.30680758e-01, 8.26140959e-03, 6.61057830e-01],
       [1.87573120e-01, 4.84271150e-04, 8.11942637e-01],
       [5.47551997e-02, 1.99111091e-05, 9.45224881e-01],
       [9.98083711e-01, 2.40170584e-06, 1.91389595e-03],
       [7.27526486e-01, 2.33722525e-03, 2.70136267e-01],
       [6.04534566e-01, 3.37156594e-01, 5.83088733e-02],
       [9.96425331e-01, 1.26918277e-03, 2.30548321e-03],
       [9.90909159e-01, 1.30384774e-06, 9.08957701e-03],
       [3.22536498e-01, 1.28774

In [17]:
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
import tensorflow as tf

# Path to your image
img_path = '00e909aa-e3ae-4558-9961-336bb0f35db3___JR_FrgE.S 8593.jpg'

# Load the image with target size matching the input shape of the model
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Start timing for the entire process (preprocessing + prediction)
total_start_time = time.time()


# Start timing for preprocessing
preprocess_start_time = time.time()

# Expand dimensions to match the expected input shape of the model (1, 400, 400, 3)
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image for MobileNetV2
img_array = preprocess_input(img_array)

# End timing for preprocessing
preprocess_end_time = time.time()
preprocess_time_taken = preprocess_end_time - preprocess_start_time

# Start timing for prediction
start_time = time.time()

# Make a prediction using the MobileNetV2 model
predictions = model.predict(img_array)

# End timing for prediction
end_time = time.time()
prediction_time_taken = end_time - start_time

# End timing for the entire process
total_end_time = time.time()
total_time_taken = total_end_time - total_start_time

# Print the predictions and time taken
print(predictions)
print(f"Time taken for preprocessing: {preprocess_time_taken} seconds")
print(f"Time taken for prediction: {prediction_time_taken} seconds")
print(f"Total time taken (preprocessing + prediction): {total_time_taken} seconds")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[9.9518484e-01 4.3396721e-08 4.8151258e-03]]
Time taken for preprocessing: 0.0 seconds
Time taken for prediction: 1.4396450519561768 seconds
Total time taken (preprocessing + prediction): 1.4396450519561768 seconds
